In [1]:
import numpy as np
from sklearn.metrics.pairwise import pairwise_kernels, check_pairwise_arrays
from sklearn.metrics.pairwise import rbf_kernel as sklearn_kernel
from scipy.stats import norm, uniform
import gurobipy as gp
from gurobipy import GRB
from rkhs_utils.utils import rkhs_norm_squared, rkhs_func_exp, rkhs_func, mmd_eps, median_heuristic, compute_scenarionumber, rbf_kernel
np.set_printoptions(suppress=True)

In [2]:
# Define constraint function

def f_constraint(x, X):
    """
    Constraint function
    
    x: cp.Variable -- decision variable (dim,)
    X: ndarray -- Samples (n_samples, dim)
    """
#     f = cp.exp(0.1 * (x[0, :] + X[:,0])) + x[1, :] + X[:, 1] -10
    
    f = -X @ x +1

    return f

In [3]:
##### Sample/set problem constants
# Define constants and sample scenarios
# np.random.seed(1)
dim = 2
n_samples = 10
c = np.random.randint(1, 100, size=dim)
# X = uniform.rvs(loc=0.8, scale=1.5, size=(n_samples, dim))
X = norm.rvs(loc=1.15, scale=0.2, size=(n_samples, dim))
sigma, _ = median_heuristic(X, X)
M = 100000
alpha = 0.05
epsilon = mmd_eps(n_samples, alpha=alpha)
print(epsilon, sigma)
print(compute_scenarionumber(0.01, dim, alpha))

1.090273278057828 0.27499275746039265
11.154352350627445


In [4]:
rbf_kernel(X, X, sigma=sigma)

array([[1.        , 0.53151412, 0.62528051, 0.35999541, 0.19938271,
        0.74866369, 0.62839843, 0.50334116, 0.35846293, 0.67950991],
       [0.53151412, 1.        , 0.26278169, 0.03992274, 0.19209582,
        0.93090769, 0.1138127 , 0.12617406, 0.13094587, 0.17117288],
       [0.62528051, 0.26278169, 1.        , 0.40652767, 0.5663157 ,
        0.36830119, 0.43225644, 0.89965398, 0.89801573, 0.28302391],
       [0.35999541, 0.03992274, 0.40652767, 1.        , 0.05915412,
        0.09142467, 0.82979915, 0.62417168, 0.31946907, 0.51431725],
       [0.19938271, 0.19209582, 0.5663157 , 0.05915412, 1.        ,
        0.19192262, 0.062447  , 0.36745817, 0.65427574, 0.03548446],
       [0.74866369, 0.93090769, 0.36830119, 0.09142467, 0.19192262,
        1.        , 0.23030934, 0.20917108, 0.18486117, 0.32367482],
       [0.62839843, 0.1138127 , 0.43225644, 0.82979915, 0.062447  ,
        0.23030934, 1.        , 0.53937557, 0.26576914, 0.85955098],
       [0.50334116, 0.12617406, 0.8996539

## Exact reformulation solving with GUROBI

In [5]:
def exact_reformulation(X, c, alpha, epsilon, M, sigma):
    n_samples, dim = X.shape
#     supp = np.linspace(np.min(X)-0.2, np.max(X)+0.2, 30)
#     xx, yy = np.meshgrid(supp, supp)
#     supp_points = np.array((xx.ravel(), yy.ravel())).T
#     supp_points = np.vstack((supp_points, X))
    supp_points = X
    with gp.Env(empty=True) as env:
        env.setParam('OutputFlag', 0)
        env.start()
        with gp.Model(env=env) as model:
    #     model = gp.Model('drccp')
            x = model.addMVar(dim, name='x')
            w = model.addMVar(n_samples, name='weights')
            f0 = model.addMVar(1, name='f0')
            mu = model.addMVar((2, supp_points.shape[0]), vtype=GRB.BINARY, name='switch')
            t = model.addMVar(1, name='slack')
            s = model.addMVar(1, lb=0.01, name='rkhs_lb')

            # Set objective
            model.setObjective(c @ x, GRB.MINIMIZE)
            model.params.NonConvex = 2

            model.addConstr(f_constraint(x, supp_points) <= (1-mu[1, :])*M)
            model.addConstr(f_constraint(x, supp_points) >= -(1-mu[0, :])*M)
            model.addConstr(rkhs_func(f0, w, X, supp_points, sigma=sigma) >= mu[0, :])
            model.addConstr(rkhs_func_exp(f0, w, X, sigma=sigma) + epsilon * t <= alpha)
            model.addConstr(rkhs_norm_squared(w, X, sigma=sigma) <= t@t)
            model.addConstr(t >= 0)

            ones = np.ones(n_samples)
            model.addConstr(w @ ones >= s)
            
            ones = np.ones(2)
            for i in range(supp_points.shape[0]):
                model.addConstr(mu[:, i] @ ones == 1)

            model.optimize()

            for v in model.getVars():
                print('%s %g' % (v.varName, v.x))
            decision_variable = (np.eye(x.shape[0]) @ x).getValue()
            print("Decision var: ", decision_variable)
            print("Constraint: ", f_constraint(x, X).getValue())
            print("RKHS function: ", rkhs_func(w, X, X, sigma=sigma).getValue())
            print("RKHS function exp: ", rkhs_func_exp(w, X, sigma=sigma).getValue())
            print("RKHS norm: ", np.sqrt(rkhs_norm_squared(w, X, sigma=sigma).getValue()))
            print('Obj: %g' % model.objVal)
            return decision_variable

In [6]:
def scenario_solution(X, c, sigma):
    n_samples, dim = X.shape
    with gp.Env(empty=True) as env:
        env.setParam('OutputFlag', 0)
        env.start()
        with gp.Model(env=env) as scenario_model:
            x = scenario_model.addMVar(dim, name='x')

            # Set objective
            scenario_model.setObjective(c @ x, GRB.MINIMIZE)
            scenario_model.params.NonConvex = 2

            scenario_model.addConstr(f_constraint(x, X) <= 0)

            scenario_model.optimize()
            decision_variable = (np.eye(x.shape[0]) @ x).getValue()
            print("Decision var: ", decision_variable)
            print("Constraint: ", f_constraint(x, X).getValue())
            print('Obj: %g' % scenario_model.objVal)
            return decision_variable

In [7]:
for eps in [0, 0.1, 0.5, 1, epsilon]:
    print("EPS: ", eps)
    print("Exact solution:")
    x_exact = exact_reformulation(X, c, alpha, eps, M, sigma)
    print("Scenario solution")
    x_scen = scenario_solution(X, c, sigma)
    X_test = norm.rvs(loc=1.15, scale=0.2, size=(1000, dim))
    f_exact = f_constraint(x_exact, X_test)
    print(np.count_nonzero(f_exact > 0))
    f_scen = f_constraint(x_scen, X_test)
    print(np.count_nonzero(f_scen > 0))

EPS:  0
Exact solution:
(10,) (1,)


GurobiError: Incompatible vector length